In [1]:
import io
import tempfile
import os
import numpy as np
import pygame
import ipywidgets as widgets
from IPython.display import display, Audio
from scipy.io import wavfile

# Initialize pygame mixer
pygame.mixer.init()

# Create a temporary directory to store the output file
temp_dir = tempfile.mkdtemp()
output_file = os.path.join(temp_dir, "output.wav")

def apply_effect(input_file, effect_type, **effect_params):
    # Load input file
    sample_rate, audio_data = wavfile.read(input_file)
    
    # Convert to float32
    audio_float = audio_data.astype(np.float32) / np.iinfo(audio_data.dtype).max
    
    # Apply the selected effect
    if effect_type == "phaser":
        effect = phaser(audio_float, sample_rate, **effect_params)
    elif effect_type == "flanger":
        effect = flanger(audio_float, sample_rate, **effect_params)
    elif effect_type == "chorus":
        effect = chorus(audio_float, sample_rate, **effect_params)
    else:
        raise ValueError("Invalid effect type")
    
    # Convert back to int16
    effect = (effect * np.iinfo(np.int16).max).astype(np.int16)
    
    # Export the result
    wavfile.write(output_file, sample_rate, effect)

def phaser(audio, sample_rate, rate=1, depth=1, feedback=0.5):
    length = len(audio)
    lfo = np.sin(2 * np.pi * rate * np.arange(length) / sample_rate)
    delay = np.interp(lfo, [-1, 1], [0, depth * sample_rate / 1000])
    indices = np.arange(length) - delay
    indices = np.clip(indices, 0, length - 1).astype(int)
    delayed = audio[indices]
    output = audio + feedback * delayed
    return output / np.max(np.abs(output))

def flanger(audio, sample_rate, rate=1, depth=1, feedback=0.5):
    length = len(audio)
    lfo = np.sin(2 * np.pi * rate * np.arange(length) / sample_rate)
    delay = np.interp(lfo, [-1, 1], [0, depth * sample_rate / 1000])
    indices = np.arange(length) - delay
    indices = np.clip(indices, 0, length - 1).astype(int)
    delayed = audio[indices]
    output = audio + feedback * delayed
    return output / np.max(np.abs(output))

def chorus(audio, sample_rate, rate=1, depth=0.02, pitch_shift=20):
    length = len(audio)
    time = np.arange(length) / sample_rate
    modulated = np.interp(
        time + depth * np.sin(2 * np.pi * rate * time),
        time,
        audio
    )
    pitched = np.interp(
        time,
        time * (1 + pitch_shift / 100),
        audio
    )
    output = audio + 0.5 * modulated + 0.5 * pitched
    return output / np.max(np.abs(output))

# Create GUI elements
file_upload = widgets.FileUpload(accept='.wav', multiple=False)
effect_dropdown = widgets.Dropdown(options=['phaser', 'flanger', 'chorus'], description='Effect:')

# Effect parameters
phaser_params = {
    'rate': widgets.FloatSlider(value=1, min=0.1, max=5, step=0.1, description='Rate:'),
    'depth': widgets.FloatSlider(value=1, min=0.1, max=5, step=0.1, description='Depth:'),
    'feedback': widgets.FloatSlider(value=0.5, min=0, max=0.9, step=0.1, description='Feedback:'),
}

flanger_params = {
    'rate': widgets.FloatSlider(value=1, min=0.1, max=5, step=0.1, description='Rate:'),
    'depth': widgets.FloatSlider(value=1, min=0.1, max=5, step=0.1, description='Depth:'),
    'feedback': widgets.FloatSlider(value=0.5, min=0, max=0.9, step=0.1, description='Feedback:'),
}

chorus_params = {
    'rate': widgets.FloatSlider(value=1, min=0.1, max=5, step=0.1, description='Rate:'),
    'depth': widgets.FloatSlider(value=0.02, min=0.001, max=0.1, step=0.001, description='Depth:'),
    'pitch_shift': widgets.FloatSlider(value=20, min=-50, max=50, step=1, description='Pitch Shift:'),
}

param_widgets = {
    'phaser': widgets.VBox([widgets.Label('Phaser Parameters:')] + list(phaser_params.values())),
    'flanger': widgets.VBox([widgets.Label('Flanger Parameters:')] + list(flanger_params.values())),
    'chorus': widgets.VBox([widgets.Label('Chorus Parameters:')] + list(chorus_params.values()))
}

output = widgets.Output()

def on_effect_change(change):
    for key, widget in param_widgets.items():
        widget.layout.display = 'none'
    param_widgets[change['new']].layout.display = None

effect_dropdown.observe(on_effect_change, names='value')

apply_button = widgets.Button(description='Apply Effect')

def on_apply_button_click(b):
    with output:
        output.clear_output()
        if not file_upload.value:
            print("Please upload a WAV file first.")
            return
        
        # Save uploaded file
        file_content = file_upload.value[0].content
        with open('input.wav', 'wb') as f:
            f.write(file_content)
        
        effect_type = effect_dropdown.value
        if effect_type == 'phaser':
            params = {k: v.value for k, v in phaser_params.items()}
        elif effect_type == 'flanger':
            params = {k: v.value for k, v in flanger_params.items()}
        else:
            params = {k: v.value for k, v in chorus_params.items()}
        
        apply_effect('input.wav', effect_type, **params)
        print(f"Effect applied. Output saved to {output_file}")
        
        # Play the audio using pygame
        pygame.mixer.music.load(output_file)
        pygame.mixer.music.play()

apply_button.on_click(on_apply_button_click)

# Display GUI
display(widgets.VBox([
    widgets.Label('Upload WAV File:'),
    file_upload,
    effect_dropdown,
    widgets.HBox([param_widgets['phaser'], param_widgets['flanger'], param_widgets['chorus']]),
    apply_button,
    output
]))

# Initialize visibility
for widget in param_widgets.values():
    widget.layout.display = 'none'
param_widgets['phaser'].layout.display = None

pygame 2.6.0 (SDL 2.28.4, Python 3.12.4)
Hello from the pygame community. https://www.pygame.org/contribute.html
